```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago, Javier Ignacio Garcia Fronti
```


# Desafio de memorabilidad: Utilizando NLP

En este notebook, intentaremos resolver el problema de regresión planteado como desafío de la materia utilizando las técnicas aprendidas de NLP.

## Preparación del ambiente

### NLP

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextDataset.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextNormalizer.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/PadSequenceTransformer.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/Word2VecVectorizer.py --directory-prefix ./Utils/

In [ ]:
!pip install transformers
!pip install unidecode
!python -m spacy download es_core_news_sm

### Sets de datos

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/ground_truth.csv --directory-prefix ./Data/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/caption_features.csv --directory-prefix ./Data/Features/

## Solución

Cargamos el set de datos

In [1]:
import pandas as pd

labels = pd.read_csv('Datasets/memorability/ground_truth.csv')
cc = pd.read_csv('Datasets/memorability/caption_features.csv', names=['sequence_name','cc'], header=0)

Nos quedamos solo con las variables que nos interesan:

In [2]:
df = pd.concat([cc['cc'], labels['memorabable']], axis=1)

Dividimos nuestros datos en Train y Test

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['cc'], df['memorabable'], 
                                                    test_size=0.33)

### Resolviendo las limitaciones de BERT

BERT es un modelo con un consumo importante de memoria (escala de forma cuadrática con el número de tokens). La implementación base de BERT está limitada a 512 tokens de entrada y por lo tanto secuencias más largas deberán se ser tratadas

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.33)

In [5]:
train.shape

(442, 2)

Hasta este momento tenemos 2 dataframes, uno para entrenamiento y uno para testing donde en cada uno disponemos de la columna de texto junto con su memorabilidad. Sin embargo, la columna de texto tiene más de 512 palabras y por lo tanto BERT no podrá procesarla por completo. Existen varias formas de resolver este problema, el paper [https://arxiv.org/abs/1905.05583] comenta varias alternativas, entre ellas:
 - Utilizando un modelo que opere sobre secuencias más largas: <a href='https://huggingface.co/transformers/model_doc/reformer.html' target='_blank'>Reformer</a>, <a href='https://huggingface.co/transformers/model_doc/longformer.html' target='_blank'>Longformer</a>
 - Ejecutar BERT sobre subsecuencias más pequeñas y luego entrenar un metamodelo que tome las predicciones de cada secuencia y las combine (LSTM).
 - Dividir la secuencia en subsecuencias y retener el contexto.
 
Mostraremos como lograr 3 en este ejemplo:

#### Dividir las secuencias en subsecuencias

`split_to_sequences`:

Toma un texto de cantidad arbitraria de palabras y lo transforma en un arreglo de M textos o secuencias donde cada secuencia tiene como máximo `sequence_len` palabras. Cada secuencia comienza con `sequence_len - text_len` palabras de la secuencia anterior para poder retener el contexto de la oración, generando así un `rolling window` 

Como ejemplo, la siguiente imágen muestra un texto donde se aplicó esta transformación utilizando `sequence_len=5` y `text_len=2`

<img src='../NLP/Docs/rolling_text.png' />

In [6]:
import numpy as np

def split_to_sequences(text, text_len=150, sequence_len=200):
    assert(text_len<sequence_len)
    
    sequences = []
    l_parcial = []
    
    if len(text.split())//text_len>0:
        nb_sequences = len(text.split())//text_len
    else: 
        nb_sequences = 1
    
    for seq in range(nb_sequences):
        if seq == 0:
            l_parcial = text.split()[:sequence_len]
            sequences.append(" ".join(l_parcial))
        else:
            l_parcial = text.split()[seq*text_len:seq*text_len + sequence_len]
            sequences.append(" ".join(l_parcial))
    
    return np.array(sequences)

Verifiquemos como se aplica en uno de los textos que tenemos disponibles

In [8]:
split_to_sequences(train['cc'][6]).shape

(6,)

Apliquemoslo sobre todo el dataset:

In [9]:
train.loc[:,'cc'] = train.cc.apply(split_to_sequences)
test.loc[:,'cc'] = test.cc.apply(split_to_sequences)

/anaconda/envs/nlp-py38/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [10]:
train['cc'][0]

array(['a man in a suit and tie standing in front of a building. a man holding a cat in his hand. a black and white photo of a cat and a cat. a man in a suit and tie standing in front of a building. a close up of a black and white photo of a bird. a man laying on a bed with a cat. a close up of a person holding a cell phone. a man in a suit and tie is holding a cell phone. a man laying on a bed with a dog. a person holding a cell phone in their handa man in a suit and tie standing in front of a building. a man holding a cat in his hand. a black and white photo of a cat and a cat. a man in a suit and tie standing in front of a building. a close up of a black and white photo of a bird. a man laying on a bed with a cat. a close up of a person holding a cell phone. a man in a suit and tie is holding a cell phone. a man laying on a bed with',
       'a close up of a black and white photo of a bird. a man laying on a bed with a cat. a close up of a person holding a cell phone. a man in a sui

#### Generando un nuevo dataset para entrenar el modelo

Hasta el momento disponemos de un dataset donde una de sus columnas es un arreglo de secuencias de texto. Esta estructura de datos no puede ser utilizada con un modelo de procesamiento de texto y por lo tanto es necesario "aplanarla". Esto quiere decir que debemos convertir los elementos del arreglo en filas de nuestro dataset.

El método `explode` transforma un data frame donde una de sus columnas es un arreglo, en otro data frame donde los elementos del arreglo se transforman en filas y los restantes valores son duplicados. Este método nos ayudará en este caso a que todas las subsecuencias que se generaron de la misma secuencia reciban el mismo `memorability_score`. El efecto de explode es el siguiente:

<img src='../NLP/Docs/explode.png' />

In [11]:
train = train.explode('cc').reset_index(drop=True)
test = test.explode('cc').reset_index()

In [12]:
train.shape

(2666, 2)

Resultado final

In [13]:
train.head(10)

,cc,memorabable
0,a woman taking a selfie in a mirror. a woman i...,1
1,a mirror. a person is taking a picture of a mi...,1
2,herself in the mirror. a woman taking a selfie...,1
3,a picture of himself in the mirror. a person t...,1
4,taking a picture of himself in the mirror. a m...,1
5,a person is taking a picture of themselves in ...,1
6,a man sitting in front of a laptop computer. a...,0
7,people standing around a desk with a laptop. a...,0
8,a room. a man sitting in front of a laptop com...,0
9,laptop computer. a man sitting in front of a l...,0
